In [ ]:
# !pip install keras
# !pip install tensorflow

In [ ]:
# needed libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model

try:
    from keras.preprocessing.image import load_img
    from keras.preprocessing.image import img_to_array
except:
    from tensorflow.keras.utils import load_img
    from tensorflow.keras.utils import img_to_array
from keras.applications.mobilenet_v2 import preprocess_input

# Feature extraction

Once the ```.dcm``` images downloaded from ADNI have been pre-processed using the ```preprocessing.ipynb``` notebook, we can use a CNN pre-trained on the ImageNet dataset, available on Keras, to extract features from the images.

The extracted features are available in the ```/data/``` folder:
- ```features.csv```: features saved in ```.csv``` format
- ```features.npz```: features saved in ```.npz``` format
- ```name_class.csv```:  images' name and class, saved in the same order as the features

These data can be imported following the model available on ```load_dataframe.ipynb```.

In [ ]:
# define the path to the pre-processed images folder
data_path = "../MRI_AX/"
image_path = data_path + "ADNI_PNG/"

# define the name of the files to which the extracted features will be saved
features_file = 'features1'
classes_file = 'classes1'

In [ ]:
# get image name
import os
imgs = np.array( os.listdir( image_path ) )
imgs = np.c_[ imgs, np.zeros( len(imgs) ) ] # column 0: image name / column 1: -

# extract image class
imgs[:,1] = [ (name.split('_')[-1]).split('.')[0] for name in imgs[:,0] ]

# save classes
np.savetxt( data_path + classes_file + '.csv', imgs, delimiter=',', fmt="%s" )

In [ ]:
# load full model
extractor = MobileNetV2()

# remove output layer
extractor = Model(inputs=extractor.inputs, outputs=extractor.layers[-2].output)

extractor.summary()

In [ ]:
# build dataset for feature extraction:
# 1. load image
# 2. convert image to array
# 3. preprocess input specifically for MobileNetV2
# 4. forward propagate input image on pretrained MobileNetV2
dataset = np.array( [ [ extractor.predict(\
                                          preprocess_input(\
                                                           img_to_array(\
                                                                        load_img( image_path + imgs[i,0], target_size=(224, 224) ) \
                                                                        ).reshape( (1, 224, 224, 3) ) ) )[0],\
                       print( '\r', i, end='' ) ][0] \
                     for i in range( imgs.shape[0] ) ] )

In [ ]:
dataset.shape

In [ ]:
# save extracted features
np.savez_compressed( data_path + features_file + '.npz', dataset )
np.savetxt( data_path + features_file + '.csv', dataset, delimiter=',' )